## Imports

In [2]:
from flask import Flask, request
import pickle

import torch
import torch.nn as nn
from torch.nn import functional as F

import numpy as np
import pandas as pd

In [5]:
! wget https://github.com/futurexskill/ml-model-deployment/raw/main/tfidfmodel.pickle

--2021-04-20 12:25:29--  https://github.com/futurexskill/ml-model-deployment/raw/main/tfidfmodel.pickle
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/futurexskill/ml-model-deployment/main/tfidfmodel.pickle [following]
--2021-04-20 12:25:30--  https://raw.githubusercontent.com/futurexskill/ml-model-deployment/main/tfidfmodel.pickle
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49660 (48K) [application/octet-stream]
Saving to: ‘tfidfmodel.pickle’

tfidfmodel.pickle   100%[===================>]  48.50K  --.-KB/s    in 0.004s  

2021-04-20 12:25:30 (11.9 MB/s) - ‘tfidfmodel.pickle’ saved [49660/

## Model 

Upload PyTorch model from 02-text_classifier-pytorch.ipynb (text_classifier_pytorch)

In [7]:
!ls

sample_data  text_classifier_pytorch  tfidfmodel.pickle


In [3]:
input_size =  467
output_size = 2 # 2 classes
hidden_size = 500

In [6]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = torch.nn.Linear(input_size, hidden_size)
    self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
    self.fc3 = torch.nn.Linear(hidden_size, output_size)

  def forward(self, X):
    X = torch.relu((self.fc1(X)))
    X = torch.relu((self.fc2(X)))
    X = self.fc3(X)

    return F.log_softmax(X, dim=1)

In [8]:
model = Net()

In [11]:
model.load_state_dict(torch.load('text_classifier_pytorch'))

<All keys matched successfully>

In [12]:
# !pip install flask-ngrok

In [21]:
from flask_ngrok import run_with_ngrok

In [26]:
app = Flask(__name__)
run_with_ngrok(app)

In [27]:
with open('tfidfmodel.pickle', 'rb') as file:
  tfidf = pickle.load(file)

In [28]:
@app.route('/predict', methods=["POST"])
def customer_behavior():
    request_data = request.get_json(force=True)
    text = request_data['sentence']
    print(f'Sentence: {text}')
    text_list = []
    text_list.append(text)
    print(text_list)
    numeric_text = tfidf.transform(text_list).toarray()
    output = model(torch.from_numpy(numeric_text).float())
    print(f"Prediction: {output[0][0]}, {output[0][1]}")
    sentiment = 'unknown'
    if torch.gt(output[0][0], output[0][1]):
      print('Negative prediction')
      sentiment = 'negative'
    else:
      print('Positive prediction')
      sentiment = 'positive'
    print(f"Prediction: {sentiment}")
    return f"The prediction is {sentiment}"

In [ ]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://c07dd9d2f3b2.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [20/Apr/2021 14:15:09] "POST /predict HTTP/1.1" 200 -


Sentence: rt tinaterrywsoc  breaking novant health just announced it will also offer walk in covid   vaccine shots at all clinics wsoctv https 
['rt tinaterrywsoc  breaking novant health just announced it will also offer walk in covid   vaccine shots at all clinics wsoctv https ']
Prediction: -0.00082923355512321, -7.095466136932373
Negative prediction
Prediction: negative


...

Sentence: not quite tired yet but need to sleep soon since i have to get up in about   hours so i have time to eat drink a t https t co b befvclfi
['not quite tired yet but need to sleep soon since i have to get up in about   hours so i have time to eat drink a t https t co b befvclfi']
Prediction: -25.406822204589844, 0.0
Positive prediction
Prediction: positive


127.0.0.1 - - [20/Apr/2021 14:15:24] "POST /predict HTTP/1.1" 200 -


Sentence: rt freedoomer cloudnikki scimitar ss kiranmartis  anshsharma vijayavignesh kmrmanish   nitesamurai who i think in another   month 
['rt freedoomer cloudnikki scimitar ss kiranmartis  anshsharma vijayavignesh kmrmanish   nitesamurai who i think in another   month ']
Prediction: 0.0, -21.31149673461914
Negative prediction
Prediction: negative


127.0.0.1 - - [20/Apr/2021 14:15:24] "POST /predict HTTP/1.1" 200 -


Sentence: rt vijayabaskarofl today we received   lakh doses of covid vaccine that shall be distributed across all vaccine centres on behalf of ho 
['rt vijayabaskarofl today we received   lakh doses of covid vaccine that shall be distributed across all vaccine centres on behalf of ho ']
Prediction: -16.478851318359375, -1.1920928244535389e-07
Positive prediction
Prediction: positive
